## Analize date

In [58]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px

plt.rc("figure", figsize=(16,8))
plt.rc("font", size=14)


In [59]:
df = pd.read_csv("data.csv") #corrected argument name

In [60]:
df.head

<bound method NDFrame.head of             date  temp
0     1981-01-01  20.7
1     1981-01-02  17.9
2     1981-01-03  18.8
3     1981-01-04  14.6
4     1981-01-05  15.8
...          ...   ...
3645  1990-12-27  14.0
3646  1990-12-28  13.6
3647  1990-12-29  13.5
3648  1990-12-30  15.7
3649  1990-12-31  13.0

[3650 rows x 2 columns]>

In [61]:
# df['ma2'] = df['temp'].rolling(2).mean()
# df['ma10'] = df['temp'].rolling(10).mean()
df['ma30'] = df['temp'].rolling(30).mean()

In [81]:
# fig = px.line(df, x='date', y='temp')
fig = px.line(df, x='date', y=df.columns[1:])
fig.show()


## Model NN

In [63]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.15.0


In [64]:
# Load and prepare the data
data = pd.read_csv('data.csv', parse_dates=[0])
X = data.iloc[:, 0:1]  # Features - date
y = data.iloc[:, 1]  # Target - temperature
max_temp = max(y)

In [72]:
from sklearn.model_selection import train_test_split

# Create rolling window dataset
def create_dataset(X, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(X.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 5
X_series, y_series = create_dataset(y, time_steps)
X_series /= max_temp
y_series /= max_temp
data_train, data_test, labels_train, labels_test = train_test_split(X_series, y_series, test_size=0.20, random_state=42)


In [73]:
# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(time_steps, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(
            #   optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mean_absolute_error'])

In [74]:
# Train the model
model.fit(data_train, labels_train, epochs=100)

Epoch 1/100
92/92 [==============================] - 0s 757us/step - loss: 0.0260 - mean_absolute_error: 0.1184
Epoch 2/100
92/92 [==============================] - 0s 769us/step - loss: 0.0096 - mean_absolute_error: 0.0772
Epoch 3/100
92/92 [==============================] - 0s 721us/step - loss: 0.0093 - mean_absolute_error: 0.0759
Epoch 4/100
92/92 [==============================] - 0s 891us/step - loss: 0.0094 - mean_absolute_error: 0.0759
Epoch 5/100
92/92 [==============================] - 0s 889us/step - loss: 0.0093 - mean_absolute_error: 0.0759
Epoch 6/100
92/92 [==============================] - 0s 758us/step - loss: 0.0093 - mean_absolute_error: 0.0758
Epoch 7/100
92/92 [==============================] - 0s 707us/step - loss: 0.0092 - mean_absolute_error: 0.0755
Epoch 8/100
92/92 [==============================] - 0s 747us/step - loss: 0.0092 - mean_absolute_error: 0.0754
Epoch 9/100
92/92 [==============================] - 0s 765us/step - loss: 0.0092 - mean_absolute_error:

In [75]:
# Evaluate the model
res = model.evaluate(data_test, labels_test, verbose=2)

23/23 - 0s - loss: 0.0093 - mean_absolute_error: 0.0754 - 78ms/epoch - 3ms/step


In [76]:
predicted = model.predict(data_test)
predicted *= max_temp

23/23 [==============================] - 0s 88us/step


In [82]:
df

,date,temp,ma30
0,1981-01-01,20.7,NaN
1,1981-01-02,17.9,NaN
2,1981-01-03,18.8,NaN
3,1981-01-04,14.6,NaN
4,1981-01-05,15.8,NaN
...,...,...,...
3645,1990-12-27,14.0,14.390000
3646,1990-12-28,13.6,14.353333
3647,1990-12-29,13.5,14.313333
3648,1990-12-30,15.7,14.413333


In [92]:
labels_resized = labels_test * max_temp
# plt.plot(labels_resized)
# plt.plot(predicted)

out_df = pd.DataFrame()
out_df['date'] = df['date'][-(len(labels_resized)):]
out_df['test'] = labels_resized
out_df['predicted'] = predicted
px.line(out_df, x='date', y=out_df.columns[1:])
# out_df

## Model LSTM

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.15.0


In [4]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px

plt.rc("figure", figsize=(16,8))
plt.rc("font", size=14)


In [5]:
# Load and prepare the data
data = pd.read_csv('data.csv', parse_dates=[0])
X = data.iloc[:, 0:1]  # Features - date
y = data.iloc[:, 1]  # Target - temperature
max_temp = max(y)

In [31]:
from sklearn.model_selection import train_test_split

# Create rolling window dataset
def create_dataset(X, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(X.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 5
X_series, y_series = create_dataset(y, time_steps)
X_series /= max_temp
y_series /= max_temp
data_train, data_test, labels_train, labels_test = train_test_split(X_series, y_series, test_size=0.20, random_state=42)



data_train = data_train.reshape((data_train.shape[0], data_train.shape[1], 1))
data_test = data_test.reshape((data_test.shape[0], data_test.shape[1], 1))

In [32]:
data_train.shape

(2916, 5, 1)

In [58]:
# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(5, 1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(
              optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mean_absolute_error'])

In [59]:
# Train the model
model.fit(data_train, labels_train, epochs=100)

Epoch 1/100
92/92 [==============================] - 1s 2ms/step - loss: 0.0226 - mean_absolute_error: 0.1135
Epoch 2/100
92/92 [==============================] - 0s 2ms/step - loss: 0.0128 - mean_absolute_error: 0.0891
Epoch 3/100
92/92 [==============================] - 0s 2ms/step - loss: 0.0126 - mean_absolute_error: 0.0870
Epoch 4/100
92/92 [==============================] - 0s 2ms/step - loss: 0.0121 - mean_absolute_error: 0.0859
Epoch 5/100
92/92 [==============================] - 0s 2ms/step - loss: 0.0117 - mean_absolute_error: 0.0845
Epoch 6/100
92/92 [==============================] - 0s 2ms/step - loss: 0.0114 - mean_absolute_error: 0.0839
Epoch 7/100
92/92 [==============================] - 0s 2ms/step - loss: 0.0108 - mean_absolute_error: 0.0820
Epoch 8/100
92/92 [==============================] - 0s 2ms/step - loss: 0.0107 - mean_absolute_error: 0.0809
Epoch 9/100
92/92 [==============================] - 0s 2ms/step - loss: 0.0103 - mean_absolute_error: 0.0796
Epoch 10/1

In [60]:
# Evaluate the model
res = model.evaluate(data_test, labels_test, verbose=2)

23/23 - 0s - loss: 0.0078 - mean_absolute_error: 0.0696 - 315ms/epoch - 14ms/step


In [61]:
predicted = model.predict(data_test)
predicted *= max_temp

23/23 [==============================] - 0s 1ms/step


In [62]:
df = pd.read_csv("data.csv") #corrected argument name
df

,date,temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8
...,...,...
3645,1990-12-27,14.0
3646,1990-12-28,13.6
3647,1990-12-29,13.5
3648,1990-12-30,15.7


In [63]:
labels_resized = labels_test * max_temp
# plt.plot(labels_resized)
# plt.plot(predicted)

out_df = pd.DataFrame()
out_df['date'] = df['date'][-(len(labels_resized)):]
out_df['test'] = labels_resized
out_df['predicted'] = predicted
px.line(out_df, x='date', y=out_df.columns[1:])
# out_df

In [64]:
inputs = tf.random.normal([32, 10, 8])
lstm = tf.keras.layers.LSTM(4)
output = lstm(inputs)
print(output.shape)

lstm = tf.keras.layers.LSTM(4, return_sequences=True, return_state=True)
whole_seq_output, final_memory_state, final_carry_state = lstm(inputs)
print(whole_seq_output.shape)

print(final_memory_state.shape)

print(final_carry_state.shape)


(32, 4)
(32, 10, 4)
(32, 4)
(32, 4)
